In [28]:
import pandas as pd
from datetime import datetime, timedelta
import tqdm
import time
import requests
import statistics as stats

In [7]:
df = pd.read_csv('/Users/brian/Downloads/archive/data.csv')

/var/folders/bn/r234zn0d75qf_wgd0b0mw7s00000gn/T/ipykernel_3019/3931934747.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,26,27,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/brian/Downloads/archive/data.csv')


In [8]:
df.head()

,OBJECTID,Shape,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,b'\x00\x01\xad\x10\x00\x00\xc8\xce\n[_@^\xc0\x...,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,1730.0,0.10,A,40.036944,-121.005833,USFS,CA,63.0,6063.0,Plumas County
1,2,b'\x00\x01\xad\x10\x00\x00\xc8\xe594\xe2\x19^\...,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1530.0,0.25,A,38.933056,-120.404444,USFS,CA,61.0,6061.0,Placer County
2,3,b'\x00\x01\xad\x10\x00\x00x{\xac \x13/^\xc0@\x...,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,2024.0,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17.0,6017.0,El Dorado County
3,4,b'\x00\x01\xad\x10\x00\x00\xc8\x13u\xd7s\xfa]\...,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1400.0,0.10,A,38.559167,-119.913333,USFS,CA,3.0,6003.0,Alpine County
4,5,b'\x00\x01\xad\x10\x00\x00\xd0\x11y\xf8\xb6\xf...,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,1200.0,0.10,A,38.559167,-119.933056,USFS,CA,3.0,6003.0,Alpine County


In [9]:
df.shape

(2303566, 39)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303566 entries, 0 to 2303565
Data columns (total 39 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   OBJECTID                       int64  
 1   Shape                          object 
 2   FOD_ID                         int64  
 3   FPA_ID                         object 
 4   SOURCE_SYSTEM_TYPE             object 
 5   SOURCE_SYSTEM                  object 
 6   NWCG_REPORTING_AGENCY          object 
 7   NWCG_REPORTING_UNIT_ID         object 
 8   NWCG_REPORTING_UNIT_NAME       object 
 9   SOURCE_REPORTING_UNIT          object 
 10  SOURCE_REPORTING_UNIT_NAME     object 
 11  LOCAL_FIRE_REPORT_ID           object 
 12  LOCAL_INCIDENT_ID              object 
 13  FIRE_CODE                      object 
 14  FIRE_NAME                      object 
 15  ICS_209_PLUS_INCIDENT_JOIN_ID  object 
 16  ICS_209_PLUS_COMPLEX_JOIN_ID   object 
 17  MTBS_ID                        object 
 18  MT

In [11]:
# List of columns to be dropped based on missing data or unnecessary content 
columns_to_drop = ['Shape', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_PLUS_INCIDENT_JOIN_ID', 'ICS_209_PLUS_COMPLEX_JOIN_ID', 'MTBS_ID', 'MTBS_FIRE_NAME', 'NWCG_CAUSE_AGE_CATEGORY', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME']

In [12]:
# Dropping columns from list above
df = df.drop(columns=columns_to_drop, axis=1)

In [13]:
# Getting only those fires from California (CA)
df = df.loc[df['STATE'] == 'CA']

In [14]:
# Confirming no states exist aside from California
df['STATE'].unique()

array(['CA'], dtype=object)

In [15]:
# Looking at values for columns to see what we are working with
df['FIPS_NAME'].unique()

array(['Plumas County', 'Placer County', 'El Dorado County',
       'Alpine County', 'Amador County', nan, 'Mono County',
       'Sierra County', 'Butte County', 'Yuba County', 'Orange County',
       'Nevada County', 'Trinity County', 'Del Norte County',
       'Los Angeles County', 'San Bernardino County', 'Fresno County',
       'San Diego County', 'Santa Barbara County', 'Mariposa County',
       'Riverside County', 'Monterey County', 'Shasta County',
       'Kern County', 'Siskiyou County', 'Tuolumne County',
       'Madera County', 'Lake County', 'Lassen County', 'Ventura County',
       'Humboldt County', 'Inyo County', 'Tehama County',
       'Calaveras County', 'Modoc County', 'Tulare County',
       'Colusa County', 'San Luis Obispo County', 'Mendocino County',
       'Glenn County', 'Stanislaus County', 'Imperial County',
       'Alameda County', 'San Benito County', 'Santa Clara County',
       'Sacramento County', 'Contra Costa County', 'Merced County',
       'Napa County

In [16]:
# Seeing what object type looks like so it can be manipulated into datetime type
df['DISCOVERY_DATE'].head()

0     2/2/2005
1    5/12/2004
2    5/31/2004
3    6/28/2004
4    6/28/2004
Name: DISCOVERY_DATE, dtype: object

In [17]:
# Digging for into regarding different fire size classes and how many of each exist
print(df['FIRE_SIZE_CLASS'].value_counts())

FIRE_SIZE_CLASS
A    138489
B     96162
C     11802
D      2555
E      1389
F       946
G       538
Name: count, dtype: int64


In [18]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'])

In [19]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,NWCG_GENERAL_CAUSE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,...,Power generation/transmission/distribution,0.10,A,40.036944,-121.005833,USFS,CA,63.0,6063.0,Plumas County
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,Natural,0.25,A,38.933056,-120.404444,USFS,CA,61.0,6061.0,Placer County
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,Debris and open burning,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17.0,6017.0,El Dorado County
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,Natural,0.10,A,38.559167,-119.913333,USFS,CA,3.0,6003.0,Alpine County
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,Natural,0.10,A,38.559167,-119.933056,USFS,CA,3.0,6003.0,Alpine County


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251881 entries, 0 to 2303562
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   OBJECTID                    251881 non-null  int64         
 1   FOD_ID                      251881 non-null  int64         
 2   FPA_ID                      251881 non-null  object        
 3   SOURCE_SYSTEM_TYPE          251881 non-null  object        
 4   SOURCE_SYSTEM               251881 non-null  object        
 5   NWCG_REPORTING_AGENCY       251881 non-null  object        
 6   NWCG_REPORTING_UNIT_ID      251881 non-null  object        
 7   NWCG_REPORTING_UNIT_NAME    251881 non-null  object        
 8   SOURCE_REPORTING_UNIT       251881 non-null  object        
 9   SOURCE_REPORTING_UNIT_NAME  251881 non-null  object        
 10  COMPLEX_NAME                1224 non-null    object        
 11  FIRE_YEAR                   251881 non-null

In [21]:
df['DISCOVERY_DATE'].head()

0   2005-02-02
1   2004-05-12
2   2004-05-31
3   2004-06-28
4   2004-06-28
Name: DISCOVERY_DATE, dtype: datetime64[ns]

In [22]:
df['WEEK_BEFORE_DISCOVERY_DATE'] = df['DISCOVERY_DATE'] - timedelta(days=7)

In [23]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,WEEK_BEFORE_DISCOVERY_DATE
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,...,0.10,A,40.036944,-121.005833,USFS,CA,63.0,6063.0,Plumas County,2005-01-26
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.25,A,38.933056,-120.404444,USFS,CA,61.0,6061.0,Placer County,2004-05-05
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17.0,6017.0,El Dorado County,2004-05-24
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.10,A,38.559167,-119.913333,USFS,CA,3.0,6003.0,Alpine County,2004-06-21
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.10,A,38.559167,-119.933056,USFS,CA,3.0,6003.0,Alpine County,2004-06-21


In [24]:
test_df = df.head(5)
test_df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,WEEK_BEFORE_DISCOVERY_DATE
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,...,0.10,A,40.036944,-121.005833,USFS,CA,63.0,6063.0,Plumas County,2005-01-26
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.25,A,38.933056,-120.404444,USFS,CA,61.0,6061.0,Placer County,2004-05-05
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.10,A,38.984167,-120.735556,STATE OR PRIVATE,CA,17.0,6017.0,El Dorado County,2004-05-24
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.10,A,38.559167,-119.913333,USFS,CA,3.0,6003.0,Alpine County,2004-06-21
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,0.10,A,38.559167,-119.933056,USFS,CA,3.0,6003.0,Alpine County,2004-06-21


In [26]:
def get_api_response(lattitude, longitude, start_date, end_date):
    url = 'https://archive-api.open-meteo.com/v1/archive'
    params = {
        "latitude": lattitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
        "precipitation_unit": "inch"
    }
    response = requests.get(url, params=params).json()
    return response['daily']

In [27]:
test_api = get_api_response(40.036944, -121.005833, "2005-01-26", "2005-02-02")
test_api

{'time': ['2005-01-26',
  '2005-01-27',
  '2005-01-28',
  '2005-01-29',
  '2005-01-30',
  '2005-01-31',
  '2005-02-01',
  '2005-02-02'],
 'temperature_2m_max': [42.4, 45.9, 42.4, 43.1, 42.3, 44.1, 46.7, 43.6],
 'temperature_2m_min': [36.6, 36.0, 34.9, 29.5, 24.4, 25.8, 22.5, 24.9],
 'precipitation_sum': [1.445, 0.161, 0.941, 0.0, 0.0, 0.0, 0.0, 0.0],
 'wind_speed_10m_max': [6.2, 7.7, 12.1, 5.9, 11.1, 12.5, 15.5, 14.5],
 'wind_gusts_10m_max': [25.7, 24.4, 50.6, 20.6, 31.8, 33.1, 46.3, 42.5],
 'wind_direction_10m_dominant': [101, 145, 169, 62, 47, 48, 49, 50]}

In [29]:
new_df = pd.DataFrame()
url = 'https://archive-api.open-meteo.com/v1/archive'
params = {
    "latitude": 40.036944,
    "longitude": -121.005833,
    "start_date": "2005-01-26",
    "end_date": "2005-02-02",
    "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "mph",
    "precipitation_unit": "inch"
}
response = requests.get(url, params=params).json()
res = response['daily']
fire_date = res['time'][-1]
avg_max_temp = stats.mean(res['temperature_2m_max'])
avg_min_temp = stats.mean(res['temperature_2m_min'])
avg_precip = stats.mean(res['precipitation_sum'])
max_wind_speed = stats.mean(res['wind_speed_10m_max'])
max_wind_gusts = stats.mean(res['wind_gusts_10m_max'])
wind_dir = stats.mean(res['wind_direction_10m_dominant'])
next_row = pd.DataFrame(
    {
        'fire_start': [fire_date],
        'max_temp': [avg_max_temp],
        'min_temp': [avg_min_temp],
        'precipitation': [avg_precip],
        'wind_speed': [max_wind_speed],
        'wind_gusts': [max_wind_gusts],
        'wind_direction': [wind_dir]
    }
)
new_df = pd.concat([new_df, next_row], axis='rows')

In [36]:
new_df

,fire_start,max_temp,min_temp,precipitation,wind_speed,wind_gusts,wind_direction
0,2005-02-02,43.8125,29.325,0.318375,10.6875,34.375,83.875


In [40]:
def process_row(row):
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    start_date = row['DISCOVERY_DATE']
    end_date = row['DISCOVERY_DATE'] - timedelta(days=7)
    print(f'lat: {latitude}, long: {longitude}, start: {start_date}, end: {end_date}')

In [41]:
test_df.apply(process_row, axis=1)

lat: 40.03694444, long: -121.00583333, start: 2005-02-02 00:00:00, end: 2005-01-26 00:00:00
lat: 38.93305556, long: -120.40444444, start: 2004-05-12 00:00:00, end: 2004-05-05 00:00:00
lat: 38.98416667, long: -120.73555556, start: 2004-05-31 00:00:00, end: 2004-05-24 00:00:00
lat: 38.55916667, long: -119.91333333, start: 2004-06-28 00:00:00, end: 2004-06-21 00:00:00
lat: 38.55916667, long: -119.93305556, start: 2004-06-28 00:00:00, end: 2004-06-21 00:00:00


0    None
1    None
2    None
3    None
4    None
dtype: object

In [39]:
for index, row in test_df.iterrows():
    latitude = test_df['LATITUDE']
    longitude = test_df['LONGITUDE']
    start_date = test_df['DISCOVERY_DATE']
    end_date = test_df['DISCOVERY_DATE'] - timedelta(days=7)
    print(f'Lati{latitude}, Long is {longitude}, start_date is {start_date}, end_date is {end_date}')

Lat is 0    40.036944
1    38.933056
2    38.984167
3    38.559167
4    38.559167
Name: LATITUDE, dtype: float64, Long is 0   -121.005833
1   -120.404444
2   -120.735556
3   -119.913333
4   -119.933056
Name: LONGITUDE, dtype: float64, start_date is 0   2005-02-02
1   2004-05-12
2   2004-05-31
3   2004-06-28
4   2004-06-28
Name: DISCOVERY_DATE, dtype: datetime64[ns], end_date is 0   2005-01-26
1   2004-05-05
2   2004-05-24
3   2004-06-21
4   2004-06-21
Name: DISCOVERY_DATE, dtype: datetime64[ns]
Lat is 0    40.036944
1    38.933056
2    38.984167
3    38.559167
4    38.559167
Name: LATITUDE, dtype: float64, Long is 0   -121.005833
1   -120.404444
2   -120.735556
3   -119.913333
4   -119.933056
Name: LONGITUDE, dtype: float64, start_date is 0   2005-02-02
1   2004-05-12
2   2004-05-31
3   2004-06-28
4   2004-06-28
Name: DISCOVERY_DATE, dtype: datetime64[ns], end_date is 0   2005-01-26
1   2004-05-05
2   2004-05-24
3   2004-06-21
4   2004-06-21
Name: DISCOVERY_DATE, dtype: datetime64[ns]
